# Theoretical Option Pricing


## Introduction

In the realm of options pricing, traditional models often simplify the intricate dynamics of underlying stock movements or are limited to certain constraints.

## Dynamic Programming

In this project we approach theoretical option pricing as a dynamic programming problem in which at each time step we have information $s_t \in \mathcal{S}$ such as the underlying price, time till expiry, interest rates, etc. We also have a set of actions we can take at each time step $a \in \mathcal{A}$ such as early exercise, hedging adjustment, taking no action, etc. We also have functions defining the cost/reward of taking an action $C(s_t, a_t)$ and probabilty of transitioning from one state to another $P(s_{t + dt} ~|~ s_t, a_t)$ typically independent of $a_t$ and corresponds to price movements of the underlying. Our goal therefore is to find $V^*(s_t)$ which denotes the theoretical value of an option under the assumtion that the holder of the contract takes the most optional actions during the lifetime of the contract. By the Bellman Optimality Principle, $V^*(s_t)$ can be recursively defined as:

<!-- $$
V(s_t) = \max_{a_t \in \mathcal{A}} \left\{
    
    C(s_t, a_t) + \sum_{\mathcal{S}} P(s_{t + 1} ~|~ s_t, a_t) \cdot V(s_{t + 1})
    
\right\} 
$$

or in the continuous case: -->

$$
V^*(s_t) = 
    \max_{a_t \in \mathcal{A}} 
    \left\{
        C(s_t, a_t) + \int_{\mathcal{S}} P(s_{t + dt} ~|~ s_t, a_t) \cdot V^*(s_{t + dt})  
    \right\} 
$$

## Approximate Dynamic Programming
To fight the curse of dimensionality inherent with financial problems we will be using approximate dynamic programming which leverages machine learning techniques to approximate $V(s_t)$. The goal in ADP is to parameterize $V(s_t)$ with parameters $\theta$ such that we minimize the Temporal Difference Value Error which is defined as the absolute difference between the predicted value of a state and the optimal value of the state:

$$
\overline{VE} = 
    V_\theta(s_t) - 
    \max_{a_t \in \mathcal{A}} 
    \left\{
        C(s_t, a_t) + \int_{\mathcal{S}} P(s_{t + dt} ~|~ s_t, a_t) \cdot V_\theta(s_{t + dt})  
    \right\} 
$$

We can then use this iteratively perform gradient descent or interpolation to converge towards the true value function.

---

## Dynamic Programming Formulation

Given a strike price $K$ our option's pay off after exercising at time $t$ is $\max\{0, S - K\}$. Further more since this is an American call option, we can exercise at any time $t \ge T$. Therefore at each time step we have the choice of exercising and recieving a payoff of $\max\{0, S - K\}$ or not exercising. If we dont exercise, our underlying stock moves to a new price $S_{t + 1}$ with probability $P(S_{t + 1} ~|~ S_t)$ and we have a new problem of whether or not we should exercise. This recursive relationship can be modeled with the following dynamic programm

$$
V(S, T) = \max\left\{ \max\{0, S - K\}, \int_{S_\text{min}}^{S_\text{max}}V(S_{t + dt}, T - dt) P(S_{t + dt} ~|~ S_t) \right\}
$$


## Setup

### Importing Libraries

In [ ]:
from src.dataset import *
from src.model import *
from src.utils import *
from config import Config

## Configuration

The configuration of our project is centralized in the `Config` class, which is located in the [`config.py`](config.py) file. This class serves as the hub for all critical parameters that govern the various aspects of our project, including the model architecture, dataset specifics, and training procedures.

### Key Features of `Config` Class:
- **Model Parameters**: Define the structure and behavior of the deep learning model. This includes the number of layers, type of neurons, activation functions, etc.
- **Data Parameters**: Specify the details regarding the dataset to be used. This includes the source of the chess game data, data format, preprocessing steps, etc.
- **Training Parameters**: Outline the training process, including learning rate, batch size, number of epochs, and other hyperparameters.

Feel free to delve into [`config.py`](config.py) and experiment with different configurations.

In [ ]:
config = Config()
print(config)

## Data Acquisition and Processing

### Data Source
Our data was sourced from the extensive [Lichess Database](https://database.lichess.org/), specifically focusing on the comprehensive November 2023 Dataset. This dataset contains 92,389,636 chess games recorded in Portable Game Notation (PGN) format. The PGN standard is critical for its detailed and accurate depiction of each game, providing an invaluable resource for our analysis.

### Selection Criteria

#### Player Rating
We selectively focused on players rated within the 1450-1550 ELO range, which is strategically centered around the mean Lichess rating of 1500 ELO. If we use a wide rating range, the underlying policy will be far from stationary. This would lead to our model being unable to converge to the optimal state values. By narrowing the rating range, we can focus on a more consistent level of play.

#### Game Format Consideration
The project focuses on classical chess games, a format most commonly played across online chess platforms. Despite slight variations in time controls, these games consistently showcase similar strategies and tactics, making them ideal for our purposes.

#### Outcome-Based Game Selection
We streamlined our dataset to include games that ended in either checkmate or stalemate. This selection criterion focuses on outcomes easily attributed to player strategy, thereby simplifying our reward system and enhancing its effectiveness. Other outcomes such as time forfeit and resignation sometimes are a result of factors such as poor internet connection.

### Data Representation
Each game in the dataset is represented as a sequence of tuples in the format $(S_t, R_{t+1}, S_{t+1}, T_{t+1})$, where:
- $S_t$ represents the state at time $t$, encoded as a one-hot encoded tensor of shape (12, 8, 8). Each channel in this tensor corresponds to a different type and color of chess piece on the board.
- $R_{t+1}$ is the reward at time $t+1$, assigned as +1 for a win, -1 for a loss, and 0 for a draw.
- $S_{t+1}$ denotes the state at time $t+1$.
- $T_{t+1}$ is a boolean indicating whether the state at time $t+1$ is terminal.

During training, batches of these tuples are sampled from the dataset and fed into the model.

### Data Streaming
Considering the vast size of our dataset, we implemented a streaming approach for processing PGN files. By utilizing an Iterable Dataset and Dataloader, we enable efficient sampling of transition batches without the need to load the entire dataset into memory. This method greatly improves computational efficiency and optimizes resource usage.


In [ ]:
dataset = TransitionDataset(
    path = config.data_path,
    min_elo = config.min_elo,
    max_elo = config.max_elo,
    event = config.event,
    termination = config.termination,
    reward_scale=config.reward_scale,
)

data_loader = TransitionDataLoader(
    dataset = dataset,
    batch_size = config.batch_size,
    device = config.device
)

## Model Architecture and Optimization

### Overview
Our model is structured as a Deep Q Network (DQN), utilizing the PyTorch framework. It's designed to evaluate and predict the value of different states in a chess game, mimicking the decision-making process of human players.

### Model Components
**Convolutional Layer**:
- *Configuration*: 256 filters, 3x3 kernel, 1x1 stride, 1x1 padding.
- *Purpose*: Extracts spatial features from the state representation, capturing relationships between chess pieces.

**Activation Layer**:
- *Type*: ReLU (Rectified Linear Unit).
- *Function*: Introduces non-linearity, enhancing the model's ability to represent complex patterns. Applied between all layers except the flattening and output layers.

**Flattening Layer**:
- *Role*: Converts the multi-dimensional output of the convolutional layer into a vector, preparing it for the fully connected layers.

**Fully Connected Layers**:
- *Configuration*: Two layers, each with 256 units.
- *Function*: Maps the flattened tensor to the value of each state, forming the basis of the decision-making process.

**Loss Function**:
- *Type*: Mean Squared Error (MSE).
- *Purpose*: Minimizes the TD Error between predicted and actual state-values, guiding the model towards accurate predictions.

**Optimizer**:
- *Type*: Adam.
- *Advantages*: Adapts the learning rate based on the gradient of the loss function, offering resilience against suboptimal hyperparameter choices.

### Optimization Process

During training, we sample batches of transitions from the dataset and feed them into the model. The model then predicts the value of each state in the batch. We then use the Bellman Equation to calculate the target values for each state. Finally, we use the MSE loss function to calculate the Mean Squared TD Error between the predicted and target values. The optimizer then uses this loss to update the model parameters, gradually improving the model's ability to predict state values.

In [ ]:
model = DeepValueNetwork(
    num_in_channels = config.num_in_channels,
    num_out_channels = config.num_out_channels,
    kernel_size = config.kernel_size,
    padding = config.padding,
    hidden_1_size = config.hidden_1_size,
    hidden_2_size = config.hidden_2_size,
    learning_rate = config.learning_rate,
    device = config.device,
)

## Training

In [ ]:
td_error_history = model.fit(
    data_loader = data_loader,
    discount_factor = config.discount_factor,
    num_batches = config.num_batches,
    model_path = config.model_path,
    log_path = config.log_path
)

In [ ]:
plot_error(td_error_history, 100)